Instructions to run the `finetune_runner.py` script to generate additional fine-tuning results for each of the 3 weights scenarios:

1. `random`
2. `10` (10% pre-train weights)
3. `20` (20% pre-train weights)

Data refers to the percentage of patients in the _Icentia11K_ dataset.

## Instructions

Prerequisites:

- Google Colab Pro
- V100 GPU


### Select instance

Select V100 GPU runtime.

### Clone Git repo

In [ ]:
%cd /root
! git clone https://github.com/myles-i/DLH_TransferLearning.git
%cd DLH_TransferLearning

/root
Cloning into 'DLH_TransferLearning'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 653 (delta 79), reused 78 (delta 38), pack-reused 527
Receiving objects: 100% (653/653), 4.54 MiB | 11.00 MiB/s, done.
Resolving deltas: 100% (388/388), done.
/root/DLH_TransferLearning


### Install dependencies



In [ ]:
%%capture
! pip install -r requirements.txt

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Determine path to data and job directories on Drive

Example for me:

`/content/drive/MyDrive/DLHProject`

where `DLHProject` is the name of my shortcut for the shared folder.

In [ ]:
# setup top level directory paths
PROJECT_DIR = '/content/drive/MyDrive/DLHProject'
DATA_DIR = PROJECT_DIR + '/data'
JOB_DIR = PROJECT_DIR + '/jobs'

In [ ]:
import os
dryrun = False # change this to False once you have verified everything looks good

# finetune/pretraining data paths (can vary depending on model used)
arch = "resnet18_2d"
batch_size = 128 #for ResNet2D, batch size needs to be 64 on V100
PHYSIONET_TRAIN = DATA_DIR + "/physionet_finetune_spectrogram_nodb/physionet_train.pkl"
PHYSIONET_TEST = DATA_DIR + "/physionet_finetune_spectrogram_nodb/physionet_test.pkl"
PRETRAIN_BASE =JOB_DIR + "/spectrogram/pretraining/16epochs_to_20percent_nodb"
FINETUNE_BASE =JOB_DIR + "/spectrogram/finetuning/experimental_nodb"
os.makedirs(FINETUNE_BASE, exist_ok=True)

# define which combinations of runs to do
weight_types = ["10",
                "20"]

weight_files = [PRETRAIN_BASE + "/epoch_08/model.weights",
                PRETRAIN_BASE + "/epoch_16/model.weights"]

# Loop through each weight type and corresponding weight file
for weight_type, weight_file in zip(weight_types, weight_files):
    # Assert that the required datasets exist
    assert os.path.exists(PHYSIONET_TRAIN), f"PHYSIONET_TRAIN does not exist: {PHYSIONET_TRAIN}"
    assert os.path.exists(PHYSIONET_TEST), f"PHYSIONET_TEST does not exist: {PHYSIONET_TEST}"
    if weight_type != "random":
        fullfile = weight_file + ".index"
        assert os.path.exists(fullfile), f"weight_file does not exist: {fullfile}"

    # Define the command to run
    command = (
        f"seq 10 10 100 | xargs -P 1 -I {{}} python finetune_runner.py \\\n"
        f"--arch {arch} \\\n"
        f"--weights-type {weight_type} \\\n"
        f"--weights-file {weight_file} \\\n"
        f"--job-base-dir {FINETUNE_BASE} \\\n"
        f"--train {PHYSIONET_TRAIN} \\\n"
        f"--test {PHYSIONET_TEST} \\\n"
        f"--batch-size 128 \\\n"
        f"--epochs 200 \\\n"
        f"--seed '{{}}' \\\n"
        f"--dryrun {dryrun} \\\n"
    )
    # Execute the command and stream the output here
    print(command)
    ! {command}



Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 75ms/step

Epoch 00007: f1 improved from 0.28825 to 0.53408, saving model to /content/drive/MyDrive/DLHProject/jobs/spectrogram/finetuning/experimental_nodb/finetune__20_seed40/best_model.weights
50/50 - 16s - loss: 0.5094 - acc: 0.7978 - val_loss: 0.6443 - val_acc: 0.7518 - f1: 0.5341 - 16s/epoch - 312ms/step
Epoch 8/200

4/4 [==============================] - 0s 70ms/step

Epoch 00008: f1 (0.33015) did not improve from 0.53408
50/50 - 15s - loss: 0.4933 - acc: 0.8102 - val_loss: 1.3450 - val_acc: 0.6721 - f1: 0.3302 - 15s/epoch - 297ms/step
Epoch 9/200

4/4 [==============================] - 0s 74ms/step

Epoch 00009: f1 (0.23723) did not improve from 0.53408
50/50 - 15s - loss: 0.4748 - acc: 0.8150 - val_loss: 1.9266 - val_acc: 0.6230 - f1: 0.2372 - 15s/epoch - 297ms/step
Epoch 10/200

4/4 [==============================] - 0s 75ms/step

Epoch 00010: f1 (0.39865) did not improve from 0.53408